In [ ]:
# Data Citation:
# F. Maxwell Harper and Joseph A. Konstan. 2015. The MovieLens Datasets: History and Context. ACM Transactions on
# Interactive Intelligent Systems (TiiS) 5, 4: 19:1–19:19. <https://doi.org/10.1145/2827872>

! curl http://files.grouplens.org/datasets/movielens/ml-latest-small.zip -o ml-latest-small.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  955k  100  955k    0     0   442k      0  0:00:02  0:00:02 --:--:--  442k


In [ ]:
import zipfile
with zipfile.ZipFile('ml-latest-small.zip', 'r') as zip_ref:
    zip_ref.extractall('data')

In [ ]:
# import the dataset
import pandas as pd
movies_df = pd.read_csv('data/ml-latest-small/movies.csv')
ratings_df = pd.read_csv('data/ml-latest-small/ratings.csv')

In [ ]:
print('The dimensions of movies dataframe are:', movies_df.shape,'\nThe dimensions of ratings dataframe are:', ratings_df.shape)

In [ ]:
# Checking movies_df
movies_df.head()

In [ ]:
# Checking ratings_df
ratings_df.head()

In [ ]:
# Movie ID to movie name mapping
movie_names = movies_df.set_index('movieId')['title'].to_dict()
n_users = len(ratings_df.userId.unique())
n_items = len(ratings_df.movieId.unique())
print("Number of unique users:", n_users)
print("Number of unique movies:", n_items)
print("The full rating matrix will have:", n_users*n_items, 'elements.')
print('----------')
print("Number of ratings:", len(ratings_df))
print("Therefore: ", len(ratings_df) / (n_users*n_items) * 100, '% of the matrix is filled.')
print("We have an incredibly sparse matrix to work with here.")
print("And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2")
print("You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.")
print("One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data")

Number of unique users: 610
Number of unique movies: 9724
The full rating matrix will have: 5931640 elements.
----------
Number of ratings: 100836
Therefore:  1.6999683055613624 % of the matrix is filled.
We have an incredibly sparse matrix to work with here.
And... as you can imagine, as the number of users and products grow, the number of elements will increase by n*2
You are going to need a lot of memory to work with global scale... storing a full matrix in memory would be a challenge.
One advantage here is that matrix factorization can realize the rating matrix implicitly, thus we don't need all the data


In [ ]:
import torch
import numpy as np
from torch.autograd import Variable
from tqdm import tqdm_notebook as tqdm

class MatrixFactorization(torch.nn.Module):
    def __init__(self, n_users, n_items, n_factors=20):
        super().__init__()
        # create user embeddings
        self.user_factors = torch.nn.Embedding(n_users, n_factors) # think of this as a lookup table for the input.
        # create item embeddings
        self.item_factors = torch.nn.Embedding(n_items, n_factors) # think of this as a lookup table for the input.
        self.user_factors.weight.data.uniform_(0, 0.05)
        self.item_factors.weight.data.uniform_(0, 0.05)

    def forward(self, data):
        # matrix multiplication
        users, items = data[:,0], data[:,1]
        return (self.user_factors(users)*self.item_factors(items)).sum(1)
    # def forward(self, user, item):
    # 	# matrix multiplication
    #     return (self.user_factors(user)*self.item_factors(item)).sum(1)

    def predict(self, user, item):
        return self.forward(user, item)

In [ ]:
# Creating the dataloader (necessary for PyTorch)
from torch.utils.data.dataset import Dataset
from torch.utils.data import DataLoader # package that helps transform your data to machine learning readiness

# Note: This isn't 'good' practice, in a MLops sense but we'll roll with this since the data is already loaded in memory.
class Loader(Dataset):
    def __init__(self):
        self.ratings = ratings_df.copy()

        # Extract all user IDs and movie IDs
        users = ratings_df.userId.unique()
        movies = ratings_df.movieId.unique()

        #--- Producing new continuous IDs for users and movies ---

        # Unique values : index
        self.userid2idx = {o:i for i,o in enumerate(users)}
        self.movieid2idx = {o:i for i,o in enumerate(movies)}

        # Obtained continuous ID for users and movies
        self.idx2userid = {i:o for o,i in self.userid2idx.items()}
        self.idx2movieid = {i:o for o,i in self.movieid2idx.items()}

        # return the id from the indexed values as noted in the lambda function down below.
        self.ratings.movieId = ratings_df.movieId.apply(lambda x: self.movieid2idx[x])
        self.ratings.userId = ratings_df.userId.apply(lambda x: self.userid2idx[x])


        self.x = self.ratings.drop(['rating', 'timestamp'], axis=1).values
        self.y = self.ratings['rating'].values
        self.x, self.y = torch.tensor(self.x), torch.tensor(self.y) # Transforms the data to tensors (ready for torch models.)

    def __getitem__(self, index):
        return (self.x[index], self.y[index])

    def __len__(self):
        return len(self.ratings)

In [ ]:
num_epochs = 128
cuda = torch.cuda.is_available()

print("Is running on GPU:", cuda)

model = MatrixFactorization(n_users, n_items, n_factors=8)
print(model)
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
# GPU enable if you have a GPU...
if cuda:
    model = model.cuda()

# MSE loss
loss_fn = torch.nn.MSELoss()

# ADAM optimizier
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

# Train data
train_set = Loader()
train_loader = DataLoader(train_set, 128, shuffle=True)

In [ ]:
for it in tqdm(range(num_epochs)):
    losses = []
    for x, y in train_loader:
         if cuda:
            x, y = x.cuda(), y.cuda()
            optimizer.zero_grad()
            outputs = model(x)
            loss = loss_fn(outputs.squeeze(), y.type(torch.float32))
            losses.append(loss.item())
            loss.backward()
            optimizer.step()
    print("iter #{}".format(it), "Loss:", sum(losses) / len(losses))

<ipython-input-11-dad152416852>:1: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for it in tqdm(range(num_epochs)):


  0%|          | 0/128 [00:00<?, ?it/s]

iter #0 Loss: 11.071608392115172
iter #1 Loss: 4.748986141028138
iter #2 Loss: 2.4787926032458465
iter #3 Loss: 1.7227501633203575
iter #4 Loss: 1.346457090777189
iter #5 Loss: 1.1286387574400394
iter #6 Loss: 0.9912383927760391
iter #7 Loss: 0.8998389748601139
iter #8 Loss: 0.836988726091869
iter #9 Loss: 0.792124355958803
iter #10 Loss: 0.7592082597731333
iter #11 Loss: 0.7343671414906603
iter #12 Loss: 0.7160692223664468
iter #13 Loss: 0.7011939089007789
iter #14 Loss: 0.6901669580924329
iter #15 Loss: 0.6817827623734619
iter #16 Loss: 0.6750307159148498
iter #17 Loss: 0.6695800366891822
iter #18 Loss: 0.6657370369370819
iter #19 Loss: 0.6628605643202206
iter #20 Loss: 0.6606594847921793
iter #21 Loss: 0.6589799897395415
iter #22 Loss: 0.6578223477235905
iter #23 Loss: 0.6566654328011015
iter #24 Loss: 0.6563512334212434
iter #25 Loss: 0.6554416160141756
iter #26 Loss: 0.6546999430142079
iter #27 Loss: 0.6541044364877159
iter #28 Loss: 0.6532314816858563
iter #29 Loss: 0.65216036428

In [ ]:
# By training the model, we will have tuned latent factors for movies and users.
c = 0
uw = 0
iw = 0
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.data)
        if c == 0:
          uw = param.data
          c +=1
        else:
          iw = param.data
        #print('param_data', param_data)

In [ ]:
trained_movie_embeddings = model.item_factors.weight.data.cpu().numpy()

In [ ]:
len(trained_movie_embeddings) # unique movie factor weights

9724

In [ ]:
from sklearn.cluster import KMeans
# Fit the clusters based on the movie weights
kmeans = KMeans(n_clusters=10, random_state=0).fit(trained_movie_embeddings)

In [ ]:
'''It can be seen here that the movies that are in the same cluster tend to have
similar genres. Also we know that the algorithm is unfamiliar with the movie name
and only obtained the relationships by looking at the numbers representing how
users have responded to the movie selections.'''
for cluster in range(10):
  print("Cluster #{}".format(cluster))
  movs = []
  for movidx in np.where(kmeans.labels_ == cluster)[0]:
    movid = train_set.idx2movieid[movidx]
    rat_count = ratings_df.loc[ratings_df['movieId']==movid].count()[0]
    movs.append((movie_names[movid], rat_count))
  for mov in sorted(movs, key=lambda tup: tup[1], reverse=True)[:10]:
    print("\t", mov[0])

Cluster #0
	 Forrest Gump (1994)
	 Toy Story (1995)
	 Aladdin (1992)
	 True Lies (1994)
	 Lion King, The (1994)
	 Back to the Future (1985)
	 Dances with Wolves (1990)
	 Beauty and the Beast (1991)
	 Mrs. Doubtfire (1993)
	 Babe (1995)
Cluster #1
	 Apollo 13 (1995)
	 Batman (1989)
	 Men in Black (a.k.a. MIB) (1997)
	 Stargate (1994)
	 Truman Show, The (1998)
	 Austin Powers: The Spy Who Shagged Me (1999)
	 Minority Report (2002)
	 Clear and Present Danger (1994)
	 Jumanji (1995)
	 Ferris Bueller's Day Off (1986)
Cluster #2
	 Batman & Robin (1997)
	 Godzilla (1998)
	 Superman IV: The Quest for Peace (1987)
	 Next Karate Kid, The (1994)
	 Rambo III (1988)
	 Eragon (2006)
	 Problem Child (1990)
	 Stop! Or My Mom Will Shoot (1992)
	 Animal, The (2001)
	 Queen of the Damned (2002)
Cluster #3
	 Jurassic Park (1993)
	 Independence Day (a.k.a. ID4) (1996)
	 Speed (1994)
	 Mission: Impossible (1996)
	 Mask, The (1994)
	 Pirates of the Caribbean: The Curse of the Black Pearl (2003)
	 Die Hard: W